In [16]:
import io
import numpy as np
import requests
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import os
from PIL import Image as PILImage

In [17]:
IMAGE_WIDTH = 512
IMAGE_HEIGHT = 512

# Download the model from Tensorflow Hub.
keras_layer = hub.KerasLayer('weights/1')
model = tf.keras.Sequential([keras_layer])
model.build([None, IMAGE_WIDTH, IMAGE_HEIGHT, 3])

In [18]:
location= "IMG_9296.mp4" # Location of the video
save_as= 'callmeby.mp4'           # Name of the converted video

In [19]:
cam = cv2.VideoCapture(location) 
try:
	if not os.path.exists('tmp'):
		os.makedirs('tmp')
except OSError:
	print ('Error: Creating directory of data')

currentframe = 0
while(True):
	ret,frame = cam.read()
	if ret:
		name = 'tmp/'+ str(currentframe) + '.jpg'
		cv2.imwrite(name, frame)
		currentframe += 1
	else:
		break
cam.release()
cv2.destroyAllWindows()

In [20]:
directory = 'tmp'
for filename in sorted(os.listdir(directory)):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        image = PILImage.open(f).convert('RGB')
        min_dim = min(image.size[0], image.size[1])
        image = image.resize((IMAGE_WIDTH * image.size[0] // min_dim, IMAGE_HEIGHT * image.size[1] // min_dim))
        input_data = np.expand_dims(image, axis=0)
        input_data = input_data[:, :IMAGE_WIDTH,:IMAGE_HEIGHT, :]
        input_data = input_data.astype(np.float) / 128 - 0.5
        output_data = model(input_data)
        assert(output_data.numpy().shape == (1, 512, 512))
        arr=np.squeeze(output_data)
        plt.imshow(arr)
        plt.axis('off')
        plt.savefig(f)
        plt.close()

/var/folders/0_/jlz3w3sd0rvgpp49r0h93j_c0000gn/T/ipykernel_88328/829579890.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  input_data = input_data.astype(np.float) / 128 - 0.5
2022-08-14 16:17:47.081112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [21]:
img_array = []
dir_len=len([name for name in os.listdir('tmp') if os.path.isfile(os.path.join('tmp', name))])
for i in range(0,dir_len):
    img = cv2.imread('tmp/'+str(i)+'.jpg')
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
out = cv2.VideoWriter(save_as,cv2.VideoWriter_fourcc(*'MP4V'), 30, size )
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
